In [38]:
from table import HoldemTable, OmahaTable
import hand


from exceptions import *
from utils import *
from hand import Hand
from ranker import *

In [ ]:
num = ["2", "3", "4", "5", "6", "7", "8", "9", "T", "J", "Q", "K", "A"]
suit = ["d", "c", "s", "h"]
a = [n + s for n in num for s in suit]


'2c'

In [ ]:
holdem = HoldemTable(num_players= 2, deck_type='full')


holdem.add_to_hand(1, ['Td', 'Ts'])
holdem.add_to_hand(2, ['Ad', 'As'])
flop = ['2d', '3d', '4d']
turn = ['5d']
river = ['6d']

def give_odds(game, flop, turn, river):
    #flop cards and calcs
    game.add_to_community(flop)
    community_cards, undrawn_combos = game.simulation_preparation(150000)
    flop_strength = game.hand_strength_analysis(res_arr = game.simulate_calculation(community_cards, undrawn_combos))
    
    #turn cards and calcs
    game.add_to_community(turn)
    community_cards, undrawn_combos = game.simulation_preparation(150000)
    turn_strength = game.hand_strength_analysis(res_arr = game.simulate_calculation(community_cards, undrawn_combos))

    #river cards and calcs
    game.add_to_community(river)
    win = game.view_result()
    return flop_strength, turn_strength, win

flop_odds, turn_odds, win_odds = give_odds(holdem, flop, turn, river)
print(flop_odds[1])
#turn into a vector of odds, starting from high card, with 0 being if the combination isn't possible
print(turn_odds[1])
print(win_odds)

{np.str_('One Pair'): np.float64(28.79), np.str_('Two Pairs'): np.float64(28.89), np.str_('Three of a Kind'): np.float64(5.25), np.str_('Straight'): np.float64(1.52), np.str_('Flush'): np.float64(32.63), np.str_('Full House'): np.float64(2.73), np.str_('Four of a Kind'): np.float64(0.1), np.str_('Straight Flush'): np.float64(0.1)}
{np.str_('Flush'): np.float64(97.73), np.str_('Straight Flush'): np.float64(2.27)}
Player 1, 2 ties with a Straight Flush
